In [1]:
from training_functions import make_tfr_input_fn
import tensorflow as tf
tf.__version__

'1.13.1'

Take the file pattern from [Beam_Pipelines.ipynb](Beam_Pipelines.ipynb):

In [2]:
from temp_dir import temp_dir
import os
file_pattern = os.path.join(temp_dir, "training.tfr-*")
file_pattern

'/tmp/tmpqmz6u8ab/training.tfr-*'

When we call ```train_input_fn```, we'll get a tensor that iterates through the training files and gets a new batch of records out of it each time it is being evaluated. In the end we will pass ```train_input_fn``` to the estimator, so that it can create the computational graph for the input stream within its own session and graph context.

In [3]:
train_input_fn = make_tfr_input_fn(
    filename_pattern=file_pattern,
    batch_size=1000, 
    options={'num_epochs': None,  # repeat infinitely
             'shuffle_buffer_size': 1000,
             'prefetch_buffer_size': 1000,
             'reader_num_threads': 10,
             'parser_num_threads': 10,
             'sloppy_ordering': True,
             'distribute': False})

---
Here, we take the tensors that provide the input stream and create an input layer from it, like described in detail in [Input_Functions.ipynb](Input_Functions.ipynb). Then we create a single ```Dense``` layer, that in essence provides the hypothesis function - a linear regression model for predicting the humidity from the the full $170$-dimensional input:

$$
h(\beta_1, \beta_2, \vartheta_1, \vartheta_2, \dots, \vartheta_{168} ) 
= (A_1, A_2, B_1, B_2, \dots, B_{168}) \cdot
\left( 
\begin {array} {c}
\beta_1 \\
\beta_2 \\
\vartheta_1 \\
\vartheta_2 \\
\dots \\
\vartheta_{168} 
\end{array}
\right) + C
$$

$\vartheta_i$ are the components of the one-hot encoded $168$-dimensional vector for the hour-of-the-week.

In [4]:
from training_functions import input_layer
features, measured_humidity = train_input_fn()
my_input_layer = input_layer(features)

linreg = tf.layers.Dense(name="LinReg", units=1)
hypothesis=linreg(my_input_layer)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please 

In [5]:
my_input_layer, hypothesis

(<tf.Tensor 'input_layer/concat:0' shape=(1000, 170) dtype=float32>,
 <tf.Tensor 'LinReg/BiasAdd:0' shape=(1000, 1) dtype=float32>)

When we evaluate the hypothesis, the computational graph will now *magically* draw on batch of 1000 records from the given input file, pass it through to the hypothesis function to arrive at 1000 1-dimensional predicions - one prediction for each of the records in the current batch.

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    out1 = sess.run(hypothesis)

In [7]:
out1.shape

(1000, 1)

---
We can actually see the variables used in the ```Dense``` layer used in the hyptothesis:

In [8]:
variables = hypothesis.graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
kernel, bias = variables[:2]

In [9]:
with tf.Session() as sess:
    sess.run(init)
    A170 = sess.run(kernel)
A170[2:].reshape([7,24])

array([[ 0.06521434, -0.10402642, -0.01323134,  0.10595527,  0.1328049 ,
        -0.18131183,  0.14806017,  0.15465802, -0.12350269, -0.00719681,
        -0.04171449, -0.04029652,  0.04115351,  0.16573974, -0.06483188,
        -0.14765114, -0.07039288,  0.17773941,  0.17429826, -0.07648033,
        -0.00978351,  0.02678879,  0.07133216, -0.0317212 ],
       [ 0.01109928,  0.03211939, -0.07110011,  0.10024449, -0.12145597,
        -0.15028816, -0.10167409,  0.10753536,  0.16905141,  0.10301551,
         0.01537775, -0.09934334, -0.06032315,  0.16834912,  0.1376606 ,
         0.00285935, -0.03789967, -0.00452437, -0.09303669, -0.16325115,
         0.03105399, -0.09547923, -0.07979932, -0.15708451],
       [ 0.0215718 , -0.00790815, -0.16041337,  0.12761524,  0.18718019,
        -0.14144632, -0.18424006, -0.18566975, -0.07759835,  0.13299936,
         0.06732535,  0.1607154 , -0.00644286,  0.1681262 ,  0.15530628,
         0.00066195, -0.12191222,  0.1862557 , -0.18236813, -0.18581012,
  

---
Now, we can manually adjust these parameters such that the predictions of the hypothesis are as close as possible to the humidity values that have actually been measured. For that, we compute the sum of the squared differences between each prediction and the measured humidity. 

In [10]:
measured_humidity, hypothesis

(<tf.Tensor 'IteratorGetNext:4' shape=(1000, 1) dtype=float32>,
 <tf.Tensor 'LinReg/BiasAdd:0' shape=(1000, 1) dtype=float32>)

In [11]:
loss = tf.reduce_mean((hypothesis-measured_humidity)**2)
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>

One of the strengths of using an ML framework is that it usually provides means to easily compute gradients. With Tensorflow, it's as easy as the following:

In [12]:
grad_k, grad_b = tf.gradients(loss, [kernel, bias])

In [13]:
grad_k, grad_b

(<tf.Tensor 'gradients/LinReg/MatMul_grad/MatMul_1:0' shape=(170, 1) dtype=float32>,
 <tf.Tensor 'gradients/LinReg/BiasAdd_grad/BiasAddGrad:0' shape=(1,) dtype=float32>)

In [14]:
learning_rate = 1e-1

In [15]:
update_k = tf.assign_sub(kernel, grad_k * learning_rate)
update_b = tf.assign_sub(bias, grad_b * learning_rate)

In mathematical terms, what we do is:
$$
A \leftarrow A - \varepsilon \cdot \frac{\partial}{\partial A} 
L(\vec{\beta}, \vec{\vartheta},  \vec{A}, \vec{B}, C)
$$

$$
B \leftarrow B - \varepsilon \cdot \frac{\partial}{\partial B} 
L(\vec{\beta}, \vec{\vartheta},  \vec{A}, \vec{B}, C)
$$

$$
C \leftarrow C - \varepsilon \cdot \frac{\partial}{\partial C} 
L(\vec{\beta}, \vec{\vartheta},  \vec{A}, \vec{B}, C)
$$

with $A$,$B$ being the *weight* parameters for $\beta_i$ and $\vartheta_i$, resp., $\varepsilon$ being the learning rate, and
$L$ being the mean squared error ```loss``` as defined above. Computationally, evaluating ```update_k``` will have the *side effect* of changing the value of variables ```A``` and ```B``` and ```update_b``` will do that with ```C```.

In short: We tweek all parameters with the help of their respective gradients.

Now, all we need to do is continously evaluate the ```update_b``` and ```update_k``` tensors and take note of the monotonously decreasing loss.

Remember: The loss is the difference between the prediction and the *reality*, the smaller it is, the better is my hypothesis.

In [16]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    l = sess.run(loss)
    print(l)
    for i in range(101):
        l, k, b, _, _ = sess.run([loss, kernel, bias, update_b, update_k])
        if i % 10 == 0:
            print(l)

415.88687
410.41013
26.742102
20.246262
15.916898
14.252321
11.804247
10.278976
8.534033
8.502633
7.5925164
8.370838


### Using Optimizers
Typically in Tensorflow, we don't do the gradient update ourselves. That's done by a special breed called *optimizers*. And some of them are particularly efficient in certain areas. As an example, we see the adaptive-momentum *ADAM* optimizer below. The inner workings of it are subject to ML lessions, but if you're too curious to stop here, the following link leads you to this well-known publication.

[Kingma, Ba 2014 - Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)

In [17]:
features, measured_humidity = train_input_fn()
my_input_layer = input_layer(features)

linreg = tf.layers.Dense(name="LinReg", units=1)
hypothesis=linreg(my_input_layer)

loss = tf.reduce_mean((hypothesis-measured_humidity)**2)

In [18]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-0)
train = optimizer.minimize(loss)

In [19]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    l = sess.run(loss)
    print(l)
    for i in range(100):
        l, k, b, _ = sess.run([loss, kernel, bias, train])
    print(l)

406.83093
3.4849946
